In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
pd.options.display.max_columns=None
pd.options.display.max_rows=None
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [6]:
train = pd.read_csv("train_LZdllcl.csv")
test = pd.read_csv("test_2umaH9m.csv")

In [3]:
sample = pd.read_csv("sample_submission_M0L0uXE.csv")

In [7]:
train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [8]:
train.shape

(54808, 14)

In [9]:
test.shape

(23490, 13)

In [10]:
sample.head()

,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [11]:
#base model

In [12]:
df = pd.DataFrame({"employee_id":test.employee_id,"is_promoted":0})
df.to_csv("basemodel1.csv",index=False)

In [14]:
df = pd.DataFrame({"employee_id":test.employee_id,"is_promoted":1})
df.to_csv("basemodel2.csv",index=False)

In [15]:
combined = pd.concat([train,test],sort=False,ignore_index=True)

In [16]:
combined.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0.0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0.0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0.0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0.0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0.0


In [17]:
combined.isnull().sum()

employee_id                 0
department                  0
region                      0
education                3443
gender                      0
recruitment_channel         0
no_of_trainings             0
age                         0
previous_year_rating     5936
length_of_service           0
KPIs_met >80%               0
awards_won?                 0
avg_training_score          0
is_promoted             23490
dtype: int64

In [18]:
combined.shape

(78298, 14)

In [19]:
combined.education.mode()

0    Bachelor's
dtype: object

In [20]:
combined.education.fillna("Bachelor's",inplace=True)

In [21]:
combined.isnull().sum()

employee_id                 0
department                  0
region                      0
education                   0
gender                      0
recruitment_channel         0
no_of_trainings             0
age                         0
previous_year_rating     5936
length_of_service           0
KPIs_met >80%               0
awards_won?                 0
avg_training_score          0
is_promoted             23490
dtype: int64

In [22]:
combined.dtypes

employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted             float64
dtype: object

In [23]:
combined.previous_year_rating.mean()

3.3322185677565574

In [25]:
combined.previous_year_rating.median()

3.0

In [26]:
#right skewed

In [27]:
combined.previous_year_rating.fillna(combined.previous_year_rating.median(),inplace=True)

In [28]:
combined.isnull().sum()

employee_id                 0
department                  0
region                      0
education                   0
gender                      0
recruitment_channel         0
no_of_trainings             0
age                         0
previous_year_rating        0
length_of_service           0
KPIs_met >80%               0
awards_won?                 0
avg_training_score          0
is_promoted             23490
dtype: int64

In [30]:
hr_analytics = combined.copy()

In [32]:
hr_analytics.shape

(78298, 14)

In [33]:
train.shape,test.shape

((54808, 14), (23490, 13))

In [34]:
dummydata = pd.get_dummies(combined, drop_first=True)
dummydata.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Below Secondary,education_Master's & above,gender_m,recruitment_channel_referred,recruitment_channel_sourcing
0,65438,1,35,5.0,8,1,0,49,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,65141,1,30,5.0,4,0,0,60,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,7513,1,34,3.0,7,0,0,50,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,2542,2,39,1.0,10,0,0,50,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,48945,1,45,3.0,2,0,0,73,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [35]:
dummydata.shape

(78298, 55)

In [36]:
newtrain = dummydata.loc[0:train.shape[0]-1]
newtest = dummydata.loc[train.shape[0]:,:]

In [37]:
newtrain.shape,newtest.shape

((54808, 55), (23490, 55))

In [38]:
X = newtrain.drop("is_promoted",axis=1)
y = newtrain["is_promoted"]

In [43]:
newtest.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Below Secondary,education_Master's & above,gender_m,recruitment_channel_referred,recruitment_channel_sourcing
54808,8724,1,24,3.0,1,1,0,77,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
54809,74430,1,31,3.0,5,0,0,51,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
54810,72255,1,31,1.0,4,0,0,47,NaN,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
54811,38562,3,31,2.0,9,0,0,65,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54812,64486,1,30,4.0,7,0,0,61,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [45]:
newtest.drop("is_promoted",axis=1,inplace=True)

In [48]:
xgb_model = XGBClassifier(max_depth=10,gamma=1)
xgb_model.fit(X,y)
test_pred = xgb_model.predict(newtest)

In [49]:
test_pred

array([0, 0, 0, ..., 0, 0, 1])

In [50]:
df = pd.DataFrame({"employee_id":newtest.employee_id,"is_promoted":test_pred})
df.to_csv("xgb_model.csv",index=False)

In [54]:
rf_model = RandomForestClassifier(criterion="entropy")
rf_model.fit(X,y)

RandomForestClassifier(criterion='entropy')

In [55]:
testpred = rf_model.predict(newtest)

In [59]:
testpred.astype(int)

array([0, 0, 0, ..., 0, 0, 1])

In [60]:
testpred

array([0., 0., 0., ..., 0., 0., 1.])

In [61]:
df = pd.DataFrame({"employee_id":newtest.employee_id,"is_promoted":testpred.astype(int)})
df.to_csv("RF_model.csv",index=False)